In [ ]:
# conda activate dash; pip install holoviews -U; cd Interactive-Dashboard ; panel serve Visualisation.ipynb
# git add Visualisation.ipynb

import holoviews as hv
import hvplot.pandas
import panel as pn
import pandas as pd
import requests

from datetime import datetime

pn.extension('tabulator')

df = pd.read_csv('Database.csv')
idf = df.interactive()

class MainGraphs:

    def __init__(self):
        self.goal_lines = [(hv.HLine(100)).opts(color='pink', line_dash='dashed'), (hv.HLine(8)).opts(color='pink', line_dash='dashed'), (hv.HLine(10000)).opts(color='pink', line_dash='dashed'), 
        (hv.HLine(2)).opts(color='pink', line_dash='dashed'), (hv.HLine(550)).opts(color='pink', line_dash='dashed')]
        self.full_selections = pn.widgets.Select(groups={'Health': ['Steps', 'Steps Score', 'Active Cals',
        'Active Cals Score', 'Diet'], 'Productivity': ['Work Hours', 'Work Score', 'Reading Hours', 'Reading Score'],
        'Personal': ['Cleaning', 'Dogs', 'Self-Care']})
        self.category_selections = pn.widgets.RadioButtonGroup(options=['Overall', 'Health', 'Productivity', 'Personal'])
        self.datapoints = len(df)

class SideBar: